# Load Packages

In [ ]:
# Add any needed packages

from google.cloud import bigquery

# Setup Connection

In [ ]:
# Fill in identifiers

EHR_OPS_PROJECT_ID = ""
EXPORT_DATASET_ID = ""

# Add any other needed identifiers
EXPORT_DATASET2_ID = ""

In [ ]:
client = bigquery.Client(project=EHR_OPS_PROJECT_ID)

# Execute Queries

## FHIR Patient Resource identifiers

Description: Lists all identifiers associated with a FHIR Patient resource

In [ ]:
# Define Query 1
q1 = f"""
SELECT
  id, identifier.system,
  identifier.value, identifier.system,
FROM `{EHR_OPS_PROJECT_ID}.{EXPORT_DATASET_ID}.Patient` p,
UNNEST(p.identifier) AS identifier
--WHERE p.id IN ('my_identifier_here')
LIMIT 100
"""

In [ ]:
results1 = client.query(q1).to_dataframe()
results1

## FHIR Patients across Submissions

Description: Identifies FHIR Patients submitted across submissions

In [ ]:
# Define Query 2

q2 = f"""
    WITH med_identifier1 AS (
      SELECT DISTINCT
        md.id
      FROM `{EHR_OPS_PROJECT_ID}.{EXPORT_DATASET_ID}.Patient` md,
        UNNEST(md.identifier) local_identifier
    ),
    med_identifier2 AS (
      SELECT DISTINCT
        md.id
      FROM `{EHR_OPS_PROJECT_ID}.{EXPORT_DATASET2_ID}.Patient` md,
        UNNEST(md.identifier) local_identifier
    )
    SELECT
      id
    FROM med_identifier2 md2
    EXCEPT DISTINCT
    SELECT
      id
    FROM  med_identifier1 md

"""

In [ ]:
results2 = client.query(q2).to_dataframe()
results2